# 0.0 Imports

In [39]:
import requests
import pandas as pd
import numpy as np
import re 
from datetime import datetime

from bs4 import BeautifulSoup

# 1.0 Loading Data

In [40]:
# URL 
url = 'https://www2.hm.com/en_us/men/products/jeans.html'
    
# Parameters
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5),AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

# Request to URL 
page = requests.get( url, headers=headers )

# Beautiful Soup object
soup = BeautifulSoup( page.text, 'html.parser' )


## 1.1 Data Collection

In [41]:
# ==============================product data========================================

products = soup.find( 'ul', class_='products-listing small' )

product_list = products.find_all( 'article', class_='hm-product-item')

# product id
product_id = [p.get( 'data-articlecode' ) for p in product_list]

# product category
product_category = [p.get( 'data-category' ) for p in product_list]

# product name
product_list = products.find_all( 'a', class_='link' )
product_name = [p.get_text() for p in product_list]

# price
product_list = products.find_all( 'span', class_='price regular' )
product_price = [p.get_text() for p in product_list]

data = pd.DataFrame( [product_id, product_category, product_name, product_price] ).T
data.columns = ['product_id', 'product_category', 'product_name', 'product_price']
data.head()



,product_id,product_category,product_name,product_price
0,1024256001,men_jeans_slim,Slim Jeans,$ 19.99
1,0985159001,men_jeans_skinny,Skinny Jeans,$ 24.99
2,1008549001,men_jeans_regular,Regular Jeans,$ 24.99
3,0690449059,men_jeans_ripped,Skinny Jeans,$ 39.99
4,1024256002,men_jeans_slim,Slim Jeans,$ 19.99


In [42]:
data.shape

(36, 4)

## 1.2 Data Collection by Product

In [43]:
# empty dataframe
df_compositions = pd.DataFrame() 

# unique columns for all products
aux = []

df_pattern = pd.DataFrame( columns=['Art. No.', 'Composition', 'Fit', 'Product safety', 'Size'] )
for i in range ( len( data ) ):
    # API Requests
    url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id' ] + '.html'
    #url = 'https://www2.hm.com/en_us/productpage.' + '0985159007' + '.html'
    print ( 'Product: {}'.format( url )  )
        
    page = requests.get( url, headers=headers )

    # Beautiful Soup object
    soup = BeautifulSoup( page.text, 'html.parser' )
    
    
    
    # ================================= color name =================================
    product_list = soup.find_all( 'a', class_='filter-option miniature active' ) + soup.find_all('a', class_='filter-option miniature')
    color_name = [p.get( 'data-color') for p in product_list]

    # product id 
    product_id = [p.get( 'data-articlecode' ) for p in product_list]

    df_color = pd.DataFrame( [product_id, color_name] ).T
    df_color.columns = ['product_id', 'color_name']
    
    for j in range( len( df_color ) ):
        # API Requests
        url = 'https://www2.hm.com/en_us/productpage.' + df_color.loc[j, 'product_id' ] + '.html'
        print ( 'Color: {}'.format( url )  )
        
        page = requests.get( url, headers=headers )

        # Beautiful Soup object
        soup = BeautifulSoup( page.text, 'html.parser' )
        
       
        #================================= Product Name =================================
        product_name = soup.find_all('section', class_='product-name-price')
        product_name = product_name[0].get_text()
        #print( product_name )                                        
                                                                  #ERRO(IMPRIME NOME E PRECO)
            
              
        # ================================= Product Price =================================
        product_price = soup.find_all( 'div', class_='primary-row product-item-price' )
        #product_price = re.findall( r'\d+\.?d+', product_price[0].get_text() )[0]
        product_price = product_price[0].get_text()
        #print( product_price )                                                         
                                                                  #ERRO(REGEX NÃO DEU CERTO, APAGO O PREÇO)
        
        # ================================= composition =================================

        # HTML all data stored 
        product_composition_list = soup.find_all( 'div', class_='details-attributes-list-item' )
        product_composition = [list(filter(None, p.get_text().split( '\n' ) ) ) for p in product_composition_list]

        # Collect the size here                             
        size_text = soup.find_all('dd')[0].text
        size_text = size_text if 'cm' in size_text else pd.NA

        for i in range(len(product_composition)):
            if product_composition[i][0] == 'Size':
                product_composition[i] = ['Size', size_text]
                break
        else:
            product_composition.append(['Size', size_text])

                
        # rename columns DataFrame
        df_composition = pd.DataFrame(product_composition).T
        df_composition.columns = df_composition.iloc[0]

        # delete first row and substitution null values 
        df_composition = df_composition.iloc[1:].fillna( method='ffill' )
                
        # remove pocket lining, shell and lining 
        df_composition['Composition'] = df_composition['Composition'].replace( 'Pocket lining:', '', regex=True )
        df_composition['Composition'] = df_composition['Composition'].replace( 'Shell:', '', regex=True )
        df_composition['Composition'] = df_composition['Composition'].replace( 'Lining:', '', regex=True )
        
        # garantee the same number of columns
        df_composition = pd.concat( [ df_pattern, df_composition ], axis=0)
        
                
        df_composition = df_composition[['Art. No.', 'Composition', 'Fit', 'Product safety', 'Size']]
        
                  
        # remane columns 
        df_composition.columns = ['product_id', 'composition', 'fit', 'product_safety', 'size']
        df_compositions['product_name'] = product_name        
        df_compositions['product_price'] = product_price
        
        
        # keep new columns if it shows up 
        aux = aux + df_composition.columns.tolist()
        
                 
        # merge data color + data compostition
        df_composition = pd.merge( df_composition, df_color,how='left', on='product_id' )
        
                       
        # all products
        df_compositions = pd.concat ( [df_compositions, df_composition], axis=0 )
        
        
# join showroom data + details
df_compositions['style_id'] = df_compositions['product_id'].apply( lambda x: x[:-3] )
df_compositions['color_id'] = df_compositions['product_id'].apply( lambda x: x[-3:] )


# scrapy datetime
df_compositions['scrapy_datetime'] = datetime.now().strftime( '%Y-%m-%d %H:%M:%S')

# #data_raw
# #data_raw.to_csv(r"C:\Users\ferki\repos\python_ds_ao_dev\datasets\data_raw_star_jeans.csv", index=False)

Product: https://www2.hm.com/en_us/productpage.1024256001.html
Color: https://www2.hm.com/en_us/productpage.1024256001.html
Color: https://www2.hm.com/en_us/productpage.1024256002.html
Color: https://www2.hm.com/en_us/productpage.1024256003.html
Color: https://www2.hm.com/en_us/productpage.1024256004.html
Color: https://www2.hm.com/en_us/productpage.1024256005.html
Color: https://www2.hm.com/en_us/productpage.1024256006.html
Color: https://www2.hm.com/en_us/productpage.1024256007.html
Color: https://www2.hm.com/en_us/productpage.1024256008.html
Product: https://www2.hm.com/en_us/productpage.0985159001.html
Color: https://www2.hm.com/en_us/productpage.0985159001.html
Color: https://www2.hm.com/en_us/productpage.0985159002.html
Color: https://www2.hm.com/en_us/productpage.0985159003.html
Color: https://www2.hm.com/en_us/productpage.0985159004.html
Color: https://www2.hm.com/en_us/productpage.0985159005.html
Color: https://www2.hm.com/en_us/productpage.0985159006.html
Color: https://www2.

Color: https://www2.hm.com/en_us/productpage.1024256003.html
Color: https://www2.hm.com/en_us/productpage.1024256005.html
Color: https://www2.hm.com/en_us/productpage.1024256006.html
Color: https://www2.hm.com/en_us/productpage.1024256007.html
Color: https://www2.hm.com/en_us/productpage.1024256008.html
Product: https://www2.hm.com/en_us/productpage.0690449036.html
Color: https://www2.hm.com/en_us/productpage.0690449036.html
Color: https://www2.hm.com/en_us/productpage.0690449001.html
Color: https://www2.hm.com/en_us/productpage.0690449002.html
Color: https://www2.hm.com/en_us/productpage.0690449006.html
Color: https://www2.hm.com/en_us/productpage.0690449007.html
Color: https://www2.hm.com/en_us/productpage.0690449009.html
Color: https://www2.hm.com/en_us/productpage.0690449011.html
Color: https://www2.hm.com/en_us/productpage.0690449013.html
Color: https://www2.hm.com/en_us/productpage.0690449021.html
Color: https://www2.hm.com/en_us/productpage.0690449022.html
Color: https://www2.hm

Product: https://www2.hm.com/en_us/productpage.1013317002.html
Color: https://www2.hm.com/en_us/productpage.1013317002.html
Color: https://www2.hm.com/en_us/productpage.1013317001.html
Color: https://www2.hm.com/en_us/productpage.1013317003.html
Color: https://www2.hm.com/en_us/productpage.1013317004.html
Color: https://www2.hm.com/en_us/productpage.1013317005.html
Color: https://www2.hm.com/en_us/productpage.1013317006.html
Color: https://www2.hm.com/en_us/productpage.1013317008.html
Color: https://www2.hm.com/en_us/productpage.1013317010.html
Color: https://www2.hm.com/en_us/productpage.1013317012.html
Product: https://www2.hm.com/en_us/productpage.1024256007.html
Color: https://www2.hm.com/en_us/productpage.1024256007.html
Color: https://www2.hm.com/en_us/productpage.1024256001.html
Color: https://www2.hm.com/en_us/productpage.1024256002.html
Color: https://www2.hm.com/en_us/productpage.1024256003.html
Color: https://www2.hm.com/en_us/productpage.1024256004.html
Color: https://www2.

In [44]:
df_compositions.head()

,product_name,product_price,product_id,composition,fit,product_safety,size,color_name,style_id,color_id,scrapy_datetime
0,\n\n\nRelaxed Jeans\n\n\n\n\n\n\n\r\n ...,\n\r\n $39.99\n,1024256001,"Cotton 99%, Spandex 1%",Slim fit,NaN,"The model is 185cm/6'1"" and wears a size 31/32",Black,1024256,001,2022-09-28 20:56:24
1,\n\n\nRelaxed Jeans\n\n\n\n\n\n\n\r\n ...,\n\r\n $39.99\n,1024256001,"Polyester 65%, Cotton 35%",Slim fit,NaN,"The model is 185cm/6'1"" and wears a size 31/32",Black,1024256,001,2022-09-28 20:56:24
2,\n\n\nRelaxed Jeans\n\n\n\n\n\n\n\r\n ...,\n\r\n $39.99\n,1024256001,"Polyester 65%, Cotton 35%",Slim fit,NaN,"The model is 185cm/6'1"" and wears a size 31/32",Black,1024256,001,2022-09-28 20:56:24
3,\n\n\nRelaxed Jeans\n\n\n\n\n\n\n\r\n ...,\n\r\n $39.99\n,1024256001,"Polyester 65%, Cotton 35%",Slim fit,NaN,"The model is 185cm/6'1"" and wears a size 31/32",Black,1024256,001,2022-09-28 20:56:24
4,\n\n\nRelaxed Jeans\n\n\n\n\n\n\n\r\n ...,\n\r\n $39.99\n,1024256001,"Polyester 65%, Cotton 35%",Slim fit,NaN,"The model is 185cm/6'1"" and wears a size 31/32",Black,1024256,001,2022-09-28 20:56:24


In [45]:
len(df_compositions['product_id'].unique() )

107

In [46]:
df_compositions.shape

(1672, 11)

# 2.0 Data Cleaning

In [55]:
# product id
df_data = df_compositions.dropna( subset=['product_id'] )

# product name
df_data['product_name'] = df_data['product_name'].str.replace( '\n', '' )
df_data['product_name'] = df_data['product_name'].str.replace( '\r', '' )
df_data['product_name'] = df_data['product_name'].str.replace( '\t', '' )
df_data['product_name'] = df_data['product_name'].str.replace( '  ', '' )
df_data['product_name'] = df_data['product_name'].str.replace( 'freefit®_', '' )
df_data['product_name'] = df_data['product_name'].str.replace( '_', '' )
df_data['product_name'] = df_data['product_name'].str.replace( ' ', '_' ).str.lower()
df_data['product_name'] = df_data['product_name'].str.split('$').str[0]


# produtc price
df_data['product_price'] = df_data['product_price'].str.replace( '\n', '' )
df_data['product_price'] = df_data['product_price'].str.replace( '\r', '' )
df_data['product_price'] = df_data['product_price'].str.replace( '\t', '' )
df_data['product_price'] = df_data['product_price'].str.replace( '$', '' )
df_data['product_price'] = df_data['product_price'].astype( float )
#df_data['product_price'] = df_data['product_price'].apply( lambda x: x.replace( '$', '') ).astype( float )


#  color name
#df_data['color_name'] = df_data['color_name'].apply( lambda x: x.replace( ' ', '_').replace('/', '_').lower() 
                                               #if pd.notnull( x ) else x )
    
df_data['color_name'] = df_data['color_name'].str.replace( ' ', '_').str.lower()      #TESTAR E APAGAR OUTRA ACIMA
                                               
# fit
df_data['fit'] = df_data['fit'].apply( lambda x: x.replace( ' ', '_').lower() if pd.notnull( x ) else x )

# size number       # exclui a medida cm
df_data['size_number'] = df_data['size'].apply( lambda x: re.search( '\d{3}cm', x ).group(0) if pd.notnull( x ) else x)
df_data['size_number'] = df_data['size'].apply( lambda x: re.search( '\d+', x ).group(0) if pd.notnull( x ) else x)

# size model 
df_data['size_model'] = df_data['size'].str.extract( '(\d+/\\d+)' )

# break composition by comma
df1 = df_data['composition'].str.split( ',', expand=True ).reset_index(drop=True)


# cotton | polyester | spandex
df_ref = pd.DataFrame( index=np.arange( len( data ) ), columns=['cotton', 'polyester', 'spandex'] )

# ================================== composition ==================================================

# --------cotton-----------------
df_cotton_0 = df1.loc[df1[0].str.contains( 'Cotton', na=True ), 0]
df_cotton_0.name = 'cotton'

df_cotton_1 = df1.loc[df1[1].str.contains( 'Cotton', na=True ), 1]
df_cotton_1.name = 'cotton'

# combine 
df_cotton = df_cotton_0.combine_first( df_cotton_1 )

df_ref = pd.concat( [df_ref, df_cotton], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last' )]

# ----------polyester--------------
df_polyester_0 = df1.loc[df1[0].str.contains( 'Polyester', na=True ), 0]
df_polyester_0.name = 'polyester'

df_polyester_1 = df1.loc[df1[1].str.contains( 'Polyester', na=True ), 1]
df_polyester_1.name = 'polyester'

# combine 
df_polyester = df_polyester_0.combine_first( df_polyester_1 )

df_ref = pd.concat( [df_ref, df_polyester], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last')]

# ----------spandex--------------
df_spandex_1 = df1.loc[df1[1].str.contains( 'Spandex', na=True ), 1]
df_spandex_1.name = 'spandex'

df_spandex_2 = df1.loc[df1[2].str.contains( 'Spandex', na=True ), 2]
df_spandex_2.name = 'spandex'
                       
# df_spandex_3 = df1.loc[df1[3].str.contains( 'Spandex', na=True ), 3]
# df_spandex_3.name = 'spandex'

# combine
#df_spandex_c2 = df_spandex_1.combine_first( df_spandex_2 )
df_spandex = df_spandex_1.combine_first( df_spandex_2 )

df_ref = pd.concat( [df_ref, df_spandex], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last')]
  
# join of combine with product_id
df_aux = pd.concat( [df_data['product_id'].reset_index(drop=True), df_ref], axis=1 )
                       
                       
# format composition data 
df_aux['cotton']     = df_aux['cotton'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )
df_aux['polyester']  = df_aux['polyester'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )
df_aux['spandex']    = df_aux['spandex'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )  
                
# final join
df_aux = df_aux.groupby( 'product_id' ).max().reset_index().fillna( 0 )
df_data = pd.merge( df_data, df_aux, on='product_id', how='left' )                     
                       

# fnal join
df_aux = pd.concat( [df_data['product_id'].reset_index(drop=True), df_ref], axis=1 )
               
# drop columns  ##remove after generating the columns size_number e size_model
df_data = df_data.drop( columns=['size', 'product_safety', 'composition'], axis=1 )

# drop duplicates 
df_data = df_data.drop_duplicates()
df_data.shape

#data.head()

C:\Users\ferki\AppData\Local\Temp\ipykernel_16184\3980028424.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_data['product_price'] = df_data['product_price'].str.replace( '$', '' )


(107, 13)

In [56]:
df_data.head()

,product_name,product_price,product_id,fit,color_name,style_id,color_id,scrapy_datetime,size_number,size_model,cotton,polyester,spandex
0,relaxed_jeans,39.99,1024256001,slim_fit,black,1024256,001,2022-09-28 20:56:24,185,31/32,0.99,0.65,0.01
7,relaxed_jeans,39.99,1024256002,slim_fit,light_denim_blue,1024256,002,2022-09-28 20:56:24,189,31/32,0.99,0.65,0.01
14,relaxed_jeans,39.99,1024256003,slim_fit,light_denim_blue,1024256,003,2022-09-28 20:56:24,187,31/32,0.99,0.65,0.01
20,relaxed_jeans,39.99,1024256004,slim_fit,denim_blue,1024256,004,2022-09-28 20:56:24,NaN,NaN,0.99,0.65,0.01
27,relaxed_jeans,39.99,1024256005,slim_fit,dark_blue,1024256,005,2022-09-28 20:56:24,183,31/32,0.99,0.65,0.01


In [57]:
df_data.shape

(107, 13)

In [58]:
df_data.tail()

,product_name,product_price,product_id,fit,color_name,style_id,color_id,scrapy_datetime,size_number,size_model,cotton,polyester,spandex
1637,relaxed_jeans,39.99,0875105017,relaxed_fit,dark_denim_blue,0875105,017,2022-09-28 20:56:24,185,31/32,1.0,0.65,0.0
1644,relaxed_jeans,39.99,0875105023,relaxed_fit,black,0875105,023,2022-09-28 20:56:24,NaN,NaN,1.0,0.65,0.0
1651,relaxed_jeans,39.99,0875105024,relaxed_fit,light_denim_blue,0875105,024,2022-09-28 20:56:24,185,31/32,1.0,0.65,0.0
1658,relaxed_jeans,39.99,0875105026,relaxed_fit,light_gray,0875105,026,2022-09-28 20:56:24,NaN,NaN,1.0,0.00,0.0
1665,NaN,NaN,0875105029,relaxed_fit,black,0875105,029,2022-09-28 20:56:24,182,32/32,1.0,0.65,0.0


In [59]:
df1.head()

,0,1,2
0,Cotton 99%,Spandex 1%,None
1,Polyester 65%,Cotton 35%,None
2,Polyester 65%,Cotton 35%,None
3,Polyester 65%,Cotton 35%,None
4,Polyester 65%,Cotton 35%,None


In [94]:
df_aux.head(20)

,product_id,cotton,polyester,spandex
0,0690449001,0.99,0.00,0.01
1,0690449002,0.98,0.00,0.02
2,0690449006,1.00,0.00,0.02
3,0690449007,1.00,0.00,0.02
4,0690449009,0.99,0.00,0.01
5,0690449011,0.99,0.00,0.01
6,0690449013,0.98,1.00,0.02
7,0690449021,0.98,1.00,0.02
8,0690449022,0.98,1.00,0.02
9,0690449024,0.98,1.00,0.02


In [60]:
df_cotton.head()

0     Cotton 99%
1     Cotton 35%
2     Cotton 35%
3     Cotton 35%
4     Cotton 35%
Name: cotton, dtype: object

In [61]:
df1[0].unique()

array([' Cotton 99%', ' Polyester 65%', ' Cotton 100%', ' Polyester 63%',
       ' Cotton 98%', 'Pocket: Cotton 100%', 'Cotton 98%', 'Cotton 99%',
       ' Polyester 100%', 'Cotton 78%', 'Cotton 77%', 'Cotton 79%',
       'Cotton 80%', ' Cotton 89%', ' Polyester 80%', ' Cotton 90%',
       'Cotton 100%'], dtype=object)

In [62]:
df1[1].unique()

array([' Spandex 1%', ' Cotton 35%', None, ' Cotton 37%', ' Spandex 2%',
       ' Polyester 21%', ' Polyester 20%', ' Polyester 19%',
       ' Elastomultiester 9%', ' Cotton 20%', ' Elastomultiester 8%'],
      dtype=object)

In [63]:
df1[2].unique()

array([None, ' Spandex 1%', ' Spandex 2%'], dtype=object)

In [ ]:
(126, 14)   # no dataset anterior

In [12]:
# data
data.to_csv(r"C:\Users\ferki\repos\python_ds_ao_dev\datasets\data_star_jeans.csv", index=False)

In [13]:
data.head()

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,color_name,Fit,size_number,size_model,cotton,polyester,spandex
0,1004199002,men_jeans_skinny,skinny_cropped_jeans,29.99,2022-06-25 16:05:26,1004199,2,light_denim_blue,skinny_fit,185,NaN,0.99,0.0,0.01
1,1004199002,men_jeans_skinny,skinny_cropped_jeans,29.99,2022-06-25 16:05:26,1004199,2,black,skinny_fit,185,NaN,0.99,0.0,0.01
2,1004199002,men_jeans_skinny,skinny_cropped_jeans,29.99,2022-06-25 16:05:26,1004199,2,denim_blue,skinny_fit,185,NaN,0.99,0.0,0.01
3,1004199002,men_jeans_skinny,skinny_cropped_jeans,29.99,2022-06-25 16:05:26,1004199,2,white,skinny_fit,185,NaN,0.99,0.0,0.01
4,1004199002,men_jeans_skinny,skinny_cropped_jeans,29.99,2022-06-25 16:05:26,1004199,2,light_gray,skinny_fit,185,NaN,0.99,0.0,0.01
